In [ ]:
# # При запуске кода в colab/kaggle можно склонировать данный репозиторий 
# !git clone https://github.com/AlexanderZah/InternVL-interp.git

In [ ]:
import os
os.environ['ROOT_DIR'] = '/kaggle/working/InternVL-interp'

In [ ]:
# !pip install -U -r $ROOT_DIR/requirements.txt

In [ ]:
!pip install -U bitsandbytes

In [ ]:
import sys

sys.path.append(os.environ["ROOT_DIR"])
sys.path.append(os.environ["ROOT_DIR"] + '/methods')
sys.path.append(os.environ["ROOT_DIR"] + 'src/caption/internvl')

In [ ]:
from methods.internvl_utils import retrieve_logit_lens_internvl, load_internvl_state
from methods.algorithms import internal_confidence, internal_confidence_heatmap, internal_confidence_segmentation
from PIL import Image
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
model_state = load_internvl_state()

In [ ]:
img_path = os.path.join(os.environ["ROOT_DIR"],"images/COCO_val2014_000000004108.jpg")
caption, softmax_probs = retrieve_logit_lens_internvl(model_state, img_path, num_patches=1)

In [ ]:
class_ = 'cake'
internal_confidence(model_state['tokenizer'], softmax_probs, class_)

In [ ]:

model_name = model_state['model_name']
heatmap_data = internal_confidence_heatmap(model_state['tokenizer'], softmax_probs, class_)
num_image_embeddings = softmax_probs.shape[2]
im = plt.imshow(heatmap_data, aspect=30/num_image_embeddings, cmap='Blues', interpolation='nearest')
plt.title(f"'{class_}' probabilities")
plt.xlabel("LM Layer")
plt.ylabel("Image Embedding Index")
plt.tight_layout()
plt.clim(0, 1)
plt.colorbar()
plt.show()

In [ ]:
img = Image.open(img_path).convert("RGB")
img_width, img_height = img.size

segmentation = internal_confidence_segmentation(model_state['tokenizer'], softmax_probs, class_, 16)
segmentation_resized = (np.array(Image.fromarray(segmentation).resize((img_width, img_height), Image.BILINEAR)))

plt.imshow(img)
plt.imshow(segmentation_resized, cmap='jet', interpolation='bilinear', alpha=.5)
plt.axis('off')
plt.title(f"'{class_}' localization")
plt.tight_layout()
plt.show()